<a href="https://colab.research.google.com/github/junkyuhufs/Practice/blob/main/CL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Corpus Lingusitics Practice

based on Kyle 홈피 
https://github.com/kristopherkyle/corpus_toolkit

분석용 Zip파일 올리기

zip파일 colab에 직접업로드 후 > 파일경로 확인후 upzip

In [3]:
!unzip -qq "/content/brown_single.zip"

corpus-toolkit설치

In [ ]:
pip install corpus-toolkit

에러발생 제거

In [ ]:
!pip install spacytextblob
!python -m textblob.download_corpora
!python -m spacy download en_core_web_sm

Load, tokenize, and generate a frequency list

In [ ]:
from corpus_toolkit import corpus_tools as ct
brown_corp = ct.ldcorpus("brown_single") #load and read corpus
tok_corp = ct.tokenize(brown_corp) #tokenize corpus - by default this lemmatizes as well
brown_freq = ct.frequency(tok_corp) #creates a frequency dictionary
#note that range can be calculated instead of frequency using the argument calc = "range"

In [ ]:
ct.head(brown_freq, hits = 10) #print top 10 items

Another way:  ldcorpus() and tokenize() 이용

In [ ]:
brown_freq = ct.frequency(ct.tokenize(ct.ldcorpus("brown_single")))

In [ ]:
ct.head(brown_freq, hits = 10)

##Concordance

Generate concordance lines: concord 함수이용; 25개 램덤샘플 (예. run/ran/running/runs)

In [ ]:
conc_results1 = ct.concord(ct.tokenize(ct.ldcorpus("brown_single"),lemma = False),["run","ran","running","runs"],nhits = 10)
for x in conc_results1:
	print(x)

Generate concordance lines: collocation search추가 (예. run/ran/running/runs + quick/quickly)

In [ ]:
conc_results2 = ct.concord(ct.tokenize(ct.ldcorpus("brown_single"),lemma = False),["run","ran","running","runs"],collocates = ["quick","quickly"], nhits = 10)
for x in conc_results2:
	print(x)

정규식표현을 이용한 concordance 및 collocation 찾기

In [ ]:
conc_results3 = ct.concord(ct.tokenize(ct.ldcorpus("brown_single"),lemma = False),["run.*","ran"],collocates = ["quick.*"], nhits = 10, regex = True)
for x in conc_results3:
	print(x)

concordance 텍스트파일로 저장

In [ ]:
#write concordance lines to a file called "run_25.txt"
conc_results4 = ct.concord(ct.tokenize(ct.ldcorpus("brown_single"),lemma = False),["run","ran","running","runs"], outname = "run_25.txt")

##Create a tagged version of your corpus

2단계로 진행: tagged 코퍼스 먼저 생성 > 그 다음 tagged 코퍼스를 다시 불러와서 분석진행

In [ ]:
tagged_brown = ct.tag(ct.ldcorpus("brown_single"))
ct.write_corpus("tagged_brown_single",tagged_brown) #the first argument is the folder where the tagged files will be written

Another way:Tag 함수를 이용해서 tagged corpus 쓰기 

In [ ]:
ct.write_corpus("tagged_brown_single",ct.tag(ct.ldcorpus("brown_single")))

reload함수를 이용해서 tagged 코퍼스를 다시 부른 후 > POA 빈도추출

In [ ]:
tagged_freq = ct.frequency(ct.reload("tagged_brown_single"))
ct.head(tagged_freq, hits = 10)

##Collocation

In [ ]:
collocates = ct.collocator(ct.tokenize(ct.ldcorpus("brown_single")),"go",stat = "MI")
#stat options include: "MI", "T", "freq", "left", and "right"

ct.head(collocates, hits = 10)

##Keyness

In [ ]:
#First, generate frequency lists for each corpus
corp1freq = ct.frequency(ct.tokenize(ct.ldcorpus("corp1")))
corp2freq = ct.frequency(ct.tokenize(ct.ldcorpus("corp2")))

#then calculate Keyness
corp_key = ct.keyness(corp1freq,corp2freq, effect = "log-ratio")
ct.head(corp_key, hits = 10) #to display top hits

##N-grams

In [ ]:
trigramfreq = ct.frequency(ct.tokenize(ct.ldcorpus("brown_single"),lemma = False, ngram = 3))
ct.head(trigramfreq, hits = 10)

##Dependency bigrams

"The player kicked the ball", the main verb kicked is connected to the noun ball via a direct object relationship, wherein kicked is the head and ball is the dependent.

In [ ]:
bg_dict = ct.dep_bigram(ct.ldcorpus("brown_single"),"dobj")
ct.head(bg_dict["bi_freq"], hits = 10)
#other keys include "dep_freq", "head_freq", and "range"
#also note that the key "samples" can be used to obtain a list of sample sentences
#but, this is not compatible with the ct.head() function (see ct.dep_conc() instead)

Strength of association

The soa() function takes a dictionary generated by the dep_bigram() function

In [ ]:
soa_mi = ct.soa(bg_dict,stat = "MI")
#other stat options include: "T", "faith_dep", "faith_head","dp_dep", and "dp_head"
ct.head(soa_mi, hits = 10)

Concordance lines for dependency bigrams

In [29]:
ct.dep_conc(bg_dict["samples"],"dobj_results")